# Code for evaluating results after running inpainting models
The inpainting models tries to recreate the background  behind an object that we want to remove. To evaluate performance, we are using perceptual hasinh, structural similarity and peak signal noise ratio to compute how "realistic" the output image is. If the model is able to recreate the background in a very realistic way, it should score well in these metrics.
Note that there are two ways of evaluating performance here: 
* For unlabeled data we are using perceptual hashing and hamming distance to compute how realistic the output image is compared to the input image
* For labeled data we are using structural similarity and peak SNR to compare the output image to the "true output".

In [8]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import imagehash
from PIL import Image


## Evaluating performance on unlabeled data
Unlabeled data means that the only images we have available is the input image (where some object is present), and the output image (where the inpainting model has tried to recreate the background behind the object). We have no knowledge of what the "true background" behind the object is. Therefore, we are evaluating performance through perceptual hashing. Perceptual hashing provides a fingerprint of the image content, and a smaller hamming distance between hashes of the input and output image indicates higher realism in the generated output image.

In [ ]:
# Performance on unlabeled data is evaluated by comparing similarity between perceptual
# hashes of the input and output images. Perceptual hashes provide a fingerprint of the image 
# content, and a smaller Hamming distance between the hashes of the input and output images indicates higher realism.
def perceptual_hash(image):
    return imagehash.average_hash(Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)))

def evaluate_unlabeled(input_image_path, output_image_path):
    # Load images
    input_image = cv2.imread(input_image_path)
    output_image = cv2.imread(output_image_path)

    # Calculate perceptual hashes
    input_hash = perceptual_hash(input_image)
    output_hash = perceptual_hash(output_image)
    
    # Calculate Hamming distance
    hamming_distance = input_hash - output_hash  # Lower values indicate higher similarity
    
    return hamming_distance



## Evaluating performance on labeled data
Labeled data means that in addition to the input image and the generated output image, we have some "true output" image available. In our case, that means that we have some image available where we have physically removed the object that the inpainting models tries to remove. This means that we now know what the background behind the given object looks like. To evaluate perfomance, we are computing the structural similarity (SSIM) and peak signal to noise ratio (PSNR) between the generated output image and the true output image.

In [23]:
def evaluate_labeled(output_image_path, ground_truth_output_path):
    output_image = cv2.imread(output_image_path)
    ground_truth_image = cv2.imread(ground_truth_output_path)
    # convert gt image to same size as output image
    height,width,_ = output_image.shape
    ground_truth_image = cv2.resize(ground_truth_image,(width,height))
    # Convert images to grayscale
    output_gray = cv2.cvtColor(output_image, cv2.COLOR_BGR2GRAY)
    ground_truth_image_gray = cv2.cvtColor(ground_truth_image, cv2.COLOR_BGR2GRAY)
    
    # Calculate SSIM and PSNR
    ssim_score = ssim(ground_truth_image_gray, output_gray)
    psnr_score = psnr(ground_truth_image_gray, output_gray)
    
    return ssim_score, psnr_score

def compute_mse(output_image_path, gt_image_path,mask_path):
    # Load the images
    output_image = cv2.imread(output_image_path)
    gt_image = cv2.imread(gt_image_path)
    mask_image = cv2.imread(mask_path)
    # convert gt image to same size as output image
    height,width,_ = output_image.shape
    gt_image = cv2.resize(gt_image,(width,height))
    #mask_image = cv2.resize(mask_image,(width,height))
    # apply mask to output and gt
    output_image = output_image[:, :, :3] * (mask_image[:, :, 3:] / 255.0)
    gt_image = gt_image[:, :, :3] * (mask_image[:, :, 3:] / 255.0)
   

    # Compute MSE
    mse = np.mean((output_image - gt_image) ** 2)

    return mse

In [ ]:
# Testing
input_image_path="images/inputs/truck_input.png"
output_image_path="images/outputs/truck_output.png"
output_image_path2="images/outputs/truck_output_tight.png"

hamming1 = evaluate_unlabeled(input_image_path,output_image_path)
hamming2 = evaluate_unlabeled(input_image_path,output_image_path2)
hamming3 = evaluate_unlabeled(input_image_path,input_image_path)

print(hamming1)
print(hamming2)
print(hamming3)
 

In [24]:
for case_nr in ["case01","case02","case03", "case04","case05","case06","case07","case08","case09","case10"]:
    mask_type = "wide"
    model_type="gan" # sd

    input_image_path = "images/"+case_nr+"/"+case_nr+"_input.png"
    output_path = "images/"+case_nr+"/"+case_nr+"_"+mask_type+"_"+model_type+"_output.png"
    ground_truth_path = "images/"+case_nr+"/"+case_nr+"_gt.png"
    mask_path = "images/"+case_nr+"/"+case_nr+"_"+mask_type+"_gan_mask.png"

    # ssim_score, psnr_score = evaluate_labeled(output_path,ground_truth_path)
    # hamming= evaluate_unlabeled(input_image_path,output_path)
    
    # display(case_nr+": SSIM: "+str(ssim_score)+", PSNR: "+str(psnr_score)+", Hamming: "+str(hamming))
    mse = compute_mse(output_path, ground_truth_path,mask_path)
    display(case_nr+": MSE: "+str(mse))


ValueError: operands could not be broadcast together with shapes (800,600,3) (806,605,0) 